In [2]:
from dotenv import load_dotenv
import os

load_dotenv()

True

In [2]:
### Faiss 벡터 db 로드
from langchain.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")

vector_db = FAISS.load_local(
    "faiss_index3",
    embeddings,
    allow_dangerous_deserialization=True
)


In [4]:
retriever = vector_db.as_retriever(
    search_type='similarity',
    search_kwargs={
        'k': 10,
    })

In [33]:
from langchain.agents import initialize_agent, AgentType
from langchain_openai import ChatOpenAI
from langchain.tools import Tool
from langchain.chains import RetrievalQA
from langchain.memory import ConversationBufferMemory
from utils.corp_search import find_corp_code, get_financial_statement, get_financial_statement_wrapper
import os


# 메모리 설정 (대화 기록 유지)

# LLM 초기화
llm = ChatOpenAI(
    temperature=0,
    model="gpt-4o-mini",
    max_tokens=1000
)

# 기존 도구들을 Tool 객체로 래핑

corp_code_tool = Tool(
    name="기업코드검색",
    func=find_corp_code,
    description="회사명을 입력받아 고유 기업코드를 반환합니다. 예: '삼성전자'"
)

financial_tool = Tool(
    name="재무제표조회",
    func=get_financial_statement_wrapper,
    description="기업코드와 연도로 재무제표를 조회합니다. 예: '005930, 2023'"
)


# Retriever가 정의되어 있다고 가정 (실제로는 벡터 스토어에서 가져와야 함)
# 예시: retriever = vector_store.as_retriever()

# 회계기준서 RAG 체인 생성
retrieval_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,  # 이 부분은 실제 retriever 객체로 교체 필요
    chain_type="stuff",
    return_source_documents=True
)

# RAG 도구 래핑
retriever_tool = Tool(
    name="회계기준서검색기",
    func=retrieval_chain.run,
    description=(
        "제무제표 관련 질문이 들어오면 꼭 사용해서 회계기준서를 확인하는 도구입니다."
        "재무제표의 각 항목이 어떤 회계 기준에 따라 작성되었는지 궁금할 때 반드시 사용해야 하는 도구입니다. "
    )
)

# 모든 도구 리스트
tools = [corp_code_tool, financial_tool, retriever_tool]

# 에이전트 초기화
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,  # ✅ memory 없는 AgentType
    memory=None,
    verbose=True,
    max_iterations=2,
    early_stopping_method="generate"
)

# 실행 함수
def run_agent(query):
    try:
        return agent.run({"input": query})  # ✅ dict로 전달
    except Exception as e:
        return f"에러가 발생했습니다: {str(e)}"

In [34]:
run_agent("2023년 삼성전자의 재무상태는 회계 기준으로 봤을떼어때?")



> Entering new AgentExecutor chain...
To answer the question about Samsung Electronics' financial position in 2023 according to accounting standards, I need to first find the company's unique corporate code. Then, I can retrieve the financial statements for 2023 and check the relevant accounting standards. 

Action: 기업코드검색  
Action Input: '삼성전자'  
Observation: 00126380
Thought:I have obtained the corporate code for Samsung Electronics, which is 00126380. Now, I will retrieve the financial statements for the year 2023 using this corporate code.

Action: 재무제표조회  
Action Input: '00126380, 2023'  
Observation: 자산총계 : 296857289000000 (당기), 260083750000000 (전기), 통화: KRW
유동자산 : 68548442000000 (당기), 59062658000000 (전기), 통화: KRW
미수금 : 1910054000000 (당기), 2925006000000 (전기), 통화: KRW
선급비용 : 1349755000000 (당기), 1047900000000 (전기), 통화: KRW
현금및현금성자산 : 6061451000000 (당기), 3921593000000 (전기), 통화: KRW
매출채권 : 27363016000000 (당기), 20503223000000 (전기), 통화: KRW
재고자산 : 29338151000000 (당기), 27990007000000 

'삼성전자의 2023년 재무상태는 다음과 같습니다. 자산총계는 296,857,289,000,000 KRW, 유동자산은 68,548,442,000,000 KRW, 비유동자산은 228,308,847,000,000 KRW입니다. 자본총계는 224,787,774,000,000 KRW이며, 부채총계는 72,069,515,000,000 KRW입니다. 당기순이익은 25,397,099,000,000 KRW입니다. 이러한 수치는 회계 기준에 따라 작성된 것입니다.'

In [44]:
from langchain.prompts import PromptTemplate
from langchain.agents import create_react_agent, AgentExecutor

# Fixed custom prompt template with all required variables
custom_prompt = PromptTemplate(
    input_variables=["input", "tools", "tool_names", "agent_scratchpad"],
    template="""
You are a helpful assistant that can use tools to answer questions.

You have access to the following tools:
{tools}

Tool names: {tool_names}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question in Korean

Begin!

Question: {input}
Thought: {agent_scratchpad}
"""
)

# Create the ReAct agent
react_agent = create_react_agent(
    llm=llm,
    tools=tools,
    prompt=custom_prompt
)

# Create agent executor
agent_executor = AgentExecutor(agent=react_agent, tools=tools, verbose=True)

In [46]:
result = agent_executor.invoke({"input": "삼성전자 2024년 재무상태|"})



> Entering new AgentExecutor chain...
Action: 기업코드검색  
Action Input: '삼성전자'  00126380Action: 재무제표조회  
Action Input: '00126380, 2024'  자산총계 : 324966127000000 (당기), 296857289000000 (전기), 통화: KRW
유동자산 : 82320322000000 (당기), 68548442000000 (전기), 통화: KRW
미수금 : 3249731000000 (당기), 1910054000000 (전기), 통화: KRW
현금및현금성자산 : 1653766000000 (당기), 6061451000000 (전기), 통화: KRW
선급비용 : 1381781000000 (당기), 1349755000000 (전기), 통화: KRW
매출채권 : 33840357000000 (당기), 27363016000000 (전기), 통화: KRW
재고자산 : 29154115000000 (당기), 29338151000000 (전기), 통화: KRW
기타유동자산 : 2852581000000 (당기), 2475944000000 (전기), 통화: KRW
단기금융상품 : 10187991000000 (당기), 50071000000 (전기), 통화: KRW
비유동자산 : 242645805000000 (당기), 228308847000000 (전기), 통화: KRW
이연법인세자산 : 14333432000000 (당기), 9931358000000 (전기), 통화: KRW
무형자산 : 10496956000000 (당기), 10440211000000 (전기), 통화: KRW
종속기업, 관계기업 및 공동기업 투자 : 57427196000000 (당기), 57392438000000 (전기), 통화: KRW
당기손익-공정가치금융자산 : 0 (당기), 1000000 (전기), 통화: KRW
기타포괄손익-공정가치금융자산 : 2176346000000 (당기), 1854503000000 (전기), 

In [47]:
result

{'input': '삼성전자 2024년 재무상태|',
 'output': '삼성전자의 2024년 재무상태는 다음과 같습니다:\n- 자산총계: 324,966,127,000,000 KRW\n- 유동자산: 82,320,322,000,000 KRW\n- 비유동자산: 242,645,805,000,000 KRW\n- 자본총계: 236,396,657,000,000 KRW\n- 부채총계: 88,569,470,000,000 KRW\n- 당기순이익: 23,582,565,000,000 KRW\n\n이 외에도 다양한 재무 항목이 있으며, 필요하신 정보가 더 있다면 말씀해 주세요.'}

In [52]:
# RAG 도구 래핑
def run_retrieval_chain(query: str) -> str:
    """RetrievalQA 체인의 invoke 메서드를 사용하여 결과를 가져오고, 'result' 키의 값만 반환하는 래퍼 함수"""
    response = retrieval_chain.invoke({"query": query})
    return response.get('result', "회계기준서 검색 결과가 없습니다.")

retriever_tool = Tool(
    name="회계기준서검색기",
    func=run_retrieval_chain,
    description=(
        "회계 기준이나 특정 재무제표 항목의 회계 처리 방식에 대한 질문이 들어오면 사용하세요. "
        "예시: 'IFRS 9의 주요 내용은 무엇인가요?', '유형자산의 감가상각 기준은?'"
    )
)

# 모든 도구 리스트
tools = [corp_code_tool, financial_tool, retriever_tool]

# 에이전트 초기화 (동일)
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    memory=None,
    verbose=True,
    max_iterations=2,
    early_stopping_method="generate"
)

# Create the ReAct agent (동일)
react_agent = create_react_agent(
    llm=llm,
    tools=tools,
    prompt=custom_prompt
)

# Create agent executor (동일)
agent_executor = AgentExecutor(agent=react_agent, tools=tools, verbose=True)

# 실행 함수 (동일)
def run_agent(query):
    try:
        return agent_executor.invoke({"input": query}) # 👈 agent.run 대신 agent_executor.invoke 사용 권장
    except Exception as e:
        return f"에러가 발생했습니다: {str(e)}"

In [54]:
run_agent('삼성전자 2024년 상태는 회계기준검색기로 봤을때 금융적으로 어뗴?')



> Entering new AgentExecutor chain...
Action: 기업코드검색  
Action Input: '삼성전자'  00126380Action: 재무제표조회  
Action Input: '00126380, 2024'  자산총계 : 324966127000000 (당기), 296857289000000 (전기), 통화: KRW
유동자산 : 82320322000000 (당기), 68548442000000 (전기), 통화: KRW
미수금 : 3249731000000 (당기), 1910054000000 (전기), 통화: KRW
현금및현금성자산 : 1653766000000 (당기), 6061451000000 (전기), 통화: KRW
선급비용 : 1381781000000 (당기), 1349755000000 (전기), 통화: KRW
매출채권 : 33840357000000 (당기), 27363016000000 (전기), 통화: KRW
재고자산 : 29154115000000 (당기), 29338151000000 (전기), 통화: KRW
기타유동자산 : 2852581000000 (당기), 2475944000000 (전기), 통화: KRW
단기금융상품 : 10187991000000 (당기), 50071000000 (전기), 통화: KRW
비유동자산 : 242645805000000 (당기), 228308847000000 (전기), 통화: KRW
이연법인세자산 : 14333432000000 (당기), 9931358000000 (전기), 통화: KRW
무형자산 : 10496956000000 (당기), 10440211000000 (전기), 통화: KRW
종속기업, 관계기업 및 공동기업 투자 : 57427196000000 (당기), 57392438000000 (전기), 통화: KRW
당기손익-공정가치금융자산 : 0 (당기), 1000000 (전기), 통화: KRW
기타포괄손익-공정가치금융자산 : 2176346000000 (당기), 1854503000000 (전기), 

{'input': '삼성전자 2024년 상태는 회계기준검색기로 봤을때 금융적으로 어뗴?',
 'output': '삼성전자의 2024년 재무 상태는 다음과 같습니다. 자산총계는 324,966,127,000,000 원, 유동자산은 82,320,322,000,000 원, 비유동자산은 242,645,805,000,000 원입니다. 자본총계는 236,396,657,000,000 원이며, 부채총계는 88,569,470,000,000 원입니다. 당기순이익은 23,582,565,000,000 원으로, 전년 대비 감소하였습니다. 전반적으로 삼성전자는 안정적인 자산 구조를 유지하고 있으나, 당기순이익의 감소가 우려됩니다.'}

In [2]:
from utils.corp_search import run_flexible_rag

question = "삼성전자 재무제표 분석해줘"
response = run_flexible_rag(question)
print(response)

📥 hybrid 처리 시작
회사: 삼성전자  
연도: 2022, 2023, 2024
{'company': '삼성전자', 'year_list': ['2022', '2023', '2024']}
{'crtfc_key': '212ec23380f7ca96c6421ecb499a1f6a5e043df9', 'corp_code': '00126380', 'bsns_year': '2022', 'reprt_code': '11011', 'fs_div': 'OFS'}
{'crtfc_key': '212ec23380f7ca96c6421ecb499a1f6a5e043df9', 'corp_code': '00126380', 'bsns_year': '2023', 'reprt_code': '11011', 'fs_div': 'OFS'}
{'crtfc_key': '212ec23380f7ca96c6421ecb499a1f6a5e043df9', 'corp_code': '00126380', 'bsns_year': '2024', 'reprt_code': '11011', 'fs_div': 'OFS'}
### 삼성전자 재무제표 분석

#### 2022년
- **자산총계**: 260,083,750,000,000 KRW
  - **유동자산**: 59,062,658,000,000 KRW
  - **비유동자산**: 201,021,092,000,000 KRW
- **부채총계**: 50,667,559,000,000 KRW
  - **유동부채**: 46,086,047,000,000 KRW
  - **비유동부채**: 4,581,512,000,000 KRW
- **자본총계**: 209,416,191,000,000 KRW
  - **이익잉여금**: 204,388,016,000,000 KRW
- **수익(매출액)**: 211,867,483,000,000 KRW
- **당기순이익**: 25,418,778,000,000 KRW
- **기타포괄손익**: 613,118,000,000 KRW
- **총포괄손익**: 26,031,896,000,

In [8]:
|from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableSequence, RunnableLambda

# 회사명과 연도를 추출하는 프롬프트
extract_prompt = PromptTemplate.from_template("""
사용자의 질문에서 회사 이름과 연도를 추출해 주세요.
사용자 질문에 따로 연도 관련 내용이 없으면 2024로 해주세요.
형식은 반드시 다음과 같이 해주세요:
회사: <회사명>
연도: <연도(4자리 숫자)>

질문: {question}
""")

# 추출 LLM 체인
extract_chain = extract_prompt | llm | StrOutputParser()

# → 파싱된 텍스트에서 "회사", "연도" 값 추출 (딕셔너리화)
def parse_extracted_text(text: str):
    lines = text.strip().splitlines()
    result = {}
    for line in lines:
        if line.startswith("회사:"):
            result["company"] = line.replace("회사:", "").strip()
        if line.startswith("연도:"):
            result["year"] = line.replace("연도:", "").strip()
    return result

extract_parser = RunnableLambda(parse_extracted_text)


In [9]:
from langchain_core.runnables import RunnableLambda

pipeline = (
    RunnableLambda(lambda x: {"question": x}) |
    extract_chain |
    RunnableLambda(lambda text: {
        **parse_extracted_text(text),
        "question": text.split("질문:")[-1].strip()
    }) |
    RunnableLambda(lambda x: {"corp_code": find_corp_code(x["company"]), **x}) |
    RunnableLambda(lambda x: {
        "financials": get_financial_statement_wrapper(f"{x['corp_code']}, {x['year']}"),
        **x
    }) |
    RunnableLambda(lambda x: {
        "accounting_info": retrieval_chain.invoke({"query": x["question"]}).get("result", "관련 기준서를 찾을 수 없습니다."),
        **x
    }) |
    PromptTemplate.from_template("""
다음은 {company}의 {year}년 정기 사업보고서에 포함된 재무제표 요약 정보입니다.

반드시 아래 정보에 있는 숫자 데이터를 분석해서, 회사의 재무상태를 설명해 주세요.
"정보 없음", "API 오류", "제공되지 않음" 등의 표현은 금지입니다.
숫자가 보이면 반드시 그것을 해석하고 분석해 주세요.

---

재무제표 요약:
{financials}

회계 기준서 요약:
{accounting_info}

질문:
{question}
""") |
    llm |
    StrOutputParser()
)


In [55]:
response = pipeline.invoke("2024년 삼성전자 재무 상태 어때?")
print(response)


삼성전자의 2024년 정기 사업보고서에 포함된 재무제표 요약 정보를 바탕으로 회사의 재무상태를 분석해 보겠습니다.

### 1. 자산 및 부채
- **자산총계**: 324,966,127,000,000 KRW (당기) / 296,857,289,000,000 KRW (전기)
  - 자산이 증가하였으며, 이는 회사의 성장 가능성을 나타냅니다.
  
- **유동자산**: 82,320,322,000,000 KRW (당기) / 68,548,442,000,000 KRW (전기)
  - 유동자산이 증가하여 단기적인 지급 능력이 개선되었습니다.

- **비유동자산**: 242,645,805,000,000 KRW (당기) / 228,308,847,000,000 KRW (전기)
  - 비유동자산도 증가하여 장기적인 투자와 자산의 안정성을 보여줍니다.

- **부채총계**: 88,569,470,000,000 KRW (당기) / 72,069,515,000,000 KRW (전기)
  - 부채가 증가하였으나, 자산 증가율이 부채 증가율보다 높아 재무적 안정성을 유지하고 있습니다.

### 2. 자본
- **자본총계**: 236,396,657,000,000 KRW (당기) / 224,787,774,000,000 KRW (전기)
  - 자본이 증가하여 회사의 자기자본 비율이 개선되었습니다. 이는 재무 건전성을 높이는 요소입니다.

### 3. 수익성
- **매출액**: 209,052,241,000,000 KRW (당기) / 170,374,090,000,000 KRW (전기)
  - 매출이 크게 증가하여 회사의 영업활동이 활발해졌음을 나타냅니다.

- **영업이익**: 12,361,034,000,000 KRW (당기) / -11,526,297,000,000 KRW (전기)
  - 영업이익이 흑자로 전환되어 영업 효율성이 개선되었습니다.

- **당기순이익**: 23,582,565,000,000 KRW (당기) / 25,397,099,000,000 KRW (전기)
  - 순이익은 감소했지만 여

In [26]:
from langchain.chains import LLMChain

# 1. 추출
def extract_company_and_year(question: str):
    extracted = extract_chain.invoke(question)
    company, year = parse_extracted_text(extracted)
    return company.strip(), year.strip(), question.strip()

# 2. 기업 코드 + 재무제표 + 기준서
def enrich_financial_info(company: str, year: str, question: str):
    corp_code = find_corp_code(company)
    financials = get_financial_statement_wrapper(f"{corp_code}, {year}")
    accounting_info = retrieval_chain.invoke({"query": question}).get("result", "")
    print(financials)
    return {
        "company": company,
        "year": year,
        "question": question,
        "financials": financials,
        "accounting_info": accounting_info
    }

# 1. PromptTemplate 정의
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

final_prompt = PromptTemplate.from_template("""
다음은 {company}의 {year}년 정기 사업보고서에 포함된 재무제표 요약 정보입니다.

반드시 아래 정보에 있는 숫자 데이터를 분석해서, 회사의 재무상태를 설명해 주세요.
"정보 없음", "API 오류", "제공되지 않음" 등의 표현은 금지입니다.
숫자가 보이면 반드시 그것을 해석하고 분석해 주세요.

---

재무제표 요약:
{financials}

회계 기준서 요약:
{accounting_info}

질문:
{question}
""")

# 2. RunnableSequence로 구성
final_chain = final_prompt | llm | StrOutputParser()

# 3. 실행
def pipeline_simple(question: str):
    company, year, q = extract_company_and_year(question)
    inputs = enrich_financial_info(company, year, q)
    return final_chain.invoke(inputs)


In [ ]:
print(final)

In [30]:
pipeline_simple('2024년 삼성전자 기업 재무제표')

[API 오류] 조회된 데이타가 없습니다.


'제공된 정보에는 삼성전자의 2024년 기업 재무제표에 대한 구체적인 숫자 데이터가 포함되어 있지 않습니다. 따라서 재무상태를 직접적으로 분석하기는 어렵습니다. \n\n그러나 일반적으로 기업의 재무상태를 평가할 때는 다음과 같은 주요 지표들을 고려합니다:\n\n1. **자산**: 총 자산의 규모는 기업의 성장 가능성을 나타냅니다. 자산이 증가하면 기업이 더 많은 자원을 활용할 수 있음을 의미합니다.\n\n2. **부채**: 총 부채는 기업의 재무적 위험을 나타냅니다. 부채 비율이 높으면 이자 부담이 커지고, 재무적 안정성이 낮아질 수 있습니다.\n\n3. **자본**: 자본은 기업의 순자산을 나타내며, 자본이 많을수록 기업의 재무적 안정성이 높아집니다.\n\n4. **수익성 지표**: 매출, 영업이익, 순이익 등의 수익성 지표는 기업의 운영 효율성을 평가하는 데 중요합니다.\n\n5. **유동성**: 유동비율이나 당좌비율과 같은 유동성 지표는 기업이 단기 채무를 이행할 수 있는 능력을 나타냅니다.\n\n이러한 지표들을 통해 삼성전자의 재무상태를 평가할 수 있습니다. 향후 재무제표가 제공된다면, 구체적인 수치를 바탕으로 더 깊이 있는 분석이 가능할 것입니다.'

### 각 기능 체크

In [72]:
# 1. 추출 확인
print(extract_chain.invoke({"question": "카카오의 2023년 재무상태를 알려줘"}))

# 2. 회사명/연도 파싱 확인
parsed = parse_extracted_text("회사: 카카오\n연도: 2023\n질문: 카카오의 2023년 재무상태")
print(parsed)

# 3. corp_code 조회
print(find_corp_code(parsed["company"]))

# 4. 재무제표 조회
print(get_financial_statement_wrapper(f"{corp_code},{parsed['year']}"))


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


회사: 카카오  
연도: 2023
{'company': '카카오', 'year': '2023'}
00918444
자산총계 : 455905980000000 (당기), 448424507000000 (전기), 통화: KRW
유동자산 : 195936557000000 (당기), 218470581000000 (전기), 통화: KRW
미수금 : 6633248000000 (당기), 6149209000000 (전기), 통화: KRW
선급비용 : 3366130000000 (당기), 2867823000000 (전기), 통화: KRW
현금및현금성자산 : 69080893000000 (당기), 49680710000000 (전기), 통화: KRW
단기상각후원가금융자산 : 608281000000 (당기), 414610000000 (전기), 통화: KRW
단기당기손익-공정가치금융자산 : 27112000000 (당기), 29080000000 (전기), 통화: KRW
매출채권 : 36647393000000 (당기), 35721563000000 (전기), 통화: KRW
재고자산 : 51625874000000 (당기), 52187866000000 (전기), 통화: KRW
매각예정분류자산 : 217864000000 (당기), 0 (전기), 통화: KRW
기타유동자산 : 5038838000000 (당기), 6316834000000 (전기), 통화: KRW
단기금융상품 : 22690924000000 (당기), 65102886000000 (전기), 통화: KRW
비유동자산 : 259969423000000 (당기), 229953926000000 (전기), 통화: KRW
이연법인세자산 : 10211797000000 (당기), 5101318000000 (전기), 통화: KRW
무형자산 : 22741862000000 (당기), 20217754000000 (전기), 통화: KRW
관계종속기업투자자산-지분법 : 11767444000000 (당기), 10893869000000 (전기), 통화: KRW
당기손익-공정가

In [105]:
parsed = extract_chain.invoke({"question": "삼성물산의 2023년 재무상태를 알려줘"})
print("🔍 파싱 결과:\n", parsed)
print("🔍 dict 변환:\n", parse_extracted_text(parsed))


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


🔍 파싱 결과:
 회사: 삼성물산  
연도: 2023
🔍 dict 변환:
 {'company': '삼성물산', 'year': '2023'}


In [106]:
corp_name = "삼성물산"
corp_code = find_corp_code(corp_name)
print("🏢 기업 코드:", corp_code)


🏢 기업 코드: 00126229


In [110]:
result = get_financial_statement_wrapper(f"{corp_code},2021")
print("📊 재무제표 결과:\n", result)

📊 재무제표 결과:
 [API 오류] 조회된 데이타가 없습니다.


In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

classification_prompt = PromptTemplate.from_template("""
다음 질문을 보고, 필요한 작업 유형을 분류하세요.
- 회계기준서만 필요: "accounting"
- 사업보고서만 필요: "business"
- 재무제표 API만 필요: "financial"
- 두 개 이상 필요: "hybrid"

질문: {question}
작업유형:
""")

classification_chain = LLMChain(llm=llm, prompt=classification_prompt)